In [52]:
import onnxruntime as ort
import numpy as np
import cv2
import math

i = 9
original = f"D:/Github/Block-Detection/data/external/nc{i}.png"


# --- Configuración ---
model_path = "ramp_XUnet_256.onnx"  # Ruta al modelo ONNX
image_path = original          # Imagen original
tile_size = 256                            # Tamaño de entrada del modelo
mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std = np.array([0.229, 0.224, 0.225], dtype=np.float32)

# --- Cargar modelo ---
session = ort.InferenceSession(model_path)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# --- Cargar imagen ---
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
orig_h, orig_w = img.shape[:2]

# --- Calcular número de tiles ---
tiles_x = math.ceil(orig_w / tile_size)
tiles_y = math.ceil(orig_h / tile_size)

# --- Crear máscara vacía ---
mask_full = np.zeros((orig_h, orig_w), dtype=np.uint8)

# --- Procesar cada tile ---
for y in range(tiles_y):
    for x in range(tiles_x):
        # Coordenadas del tile
        x_start = x * tile_size
        y_start = y * tile_size
        x_end = min(x_start + tile_size, orig_w)
        y_end = min(y_start + tile_size, orig_h)

        # Extraer tile y rellenar si es necesario
        tile = img[y_start:y_end, x_start:x_end]
        pad_h = tile_size - tile.shape[0]
        pad_w = tile_size - tile.shape[1]
        tile_padded = cv2.copyMakeBorder(tile, 0, pad_h, 0, pad_w,
                                         cv2.BORDER_CONSTANT, value=(0, 0, 0))

        # Normalización
        tile_norm = tile_padded.astype(np.float32) / 255.0
        tile_norm = (tile_norm - mean) / std
        tile_norm = np.transpose(tile_norm, (2, 0, 1))  # (C,H,W)
        tile_norm = np.expand_dims(tile_norm, axis=0)   # (1,C,H,W)

        # Inferencia
        outputs = session.run([output_name], {input_name: tile_norm})
        logits = outputs[0][0][0]  # (H,W)
        prob = 1 / (1 + np.exp(-logits))  # Sigmoide
        segmentation = (prob > 0.5).astype(np.uint8)

        # Recortar padding y colocar en máscara final
        segmentation_cropped = segmentation[:tile.shape[0], :tile.shape[1]]
        mask_full[y_start:y_end, x_start:x_end] = segmentation_cropped

# --- Guardar máscara recombinada ---
cv2.imwrite("mask_recombinada.png", mask_full * 255)
print("✅ Máscara recombinada guardada en mask_recombinada.png")

✅ Máscara recombinada guardada en mask_recombinada.png


In [53]:
import cv2
import numpy as np

# Cargar imagen original y máscara
img_orig = cv2.imread(original)
mask = cv2.imread('mask_recombinada.png', cv2.IMREAD_GRAYSCALE)

# Crear overlay en color (ej. rojo para edificios)
overlay = img_orig.copy()
overlay[mask == 0] = [0, 0, 255]  # píxeles rojos donde hay edificios

# Combinar con transparencia
alpha = 0.4
superpuesta = cv2.addWeighted(overlay, alpha, img_orig, 1 - alpha, 0)

cv2.imwrite(f"D:/Github/Block-Detection/data/external/xunet/xnet{i}.png", superpuesta)
print("✅ Imagen superpuesta guardada en comparacion.png")

✅ Imagen superpuesta guardada en comparacion.png
